In [2]:
from glob import glob
import xarray as xr
import cftime
import nc_time_axis
import numpy as np
import matplotlib.pyplot as plt
import intake, intake_esm

## Read the CM4 data

In [3]:
#col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json"
col_url = "https://raw.githubusercontent.com/aradhakrishnanGFDL/gfdl-aws-analysis/community/esm-collection-spec-examples/esgf-world.json"

In [4]:
col = intake.open_esm_datastore(col_url)

In [5]:
col.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
0,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,AERmon,r1i1p1f1,gn,ps,185001-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
1,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,CFmon,r1i1p1f1,gn,ta,185001-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
2,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,LImon,r1i1p1f1,gn,snc,185002-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
3,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,LImon,r1i1p1f1,gn,snd,185002-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
4,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,LImon,r1i1p1f1,gn,snw,185002-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
867998,CMIP6,UA,MCM-UA-1-0,ssp585,mon,atmos,Amon,r1i1p1f2,gn,rlut,201501-210012,v20190731,s3://esgf-world/CMIP6/ScenarioMIP/UA/MCM-UA-1-...
867999,CMIP6,UA,MCM-UA-1-0,ssp585,mon,atmos,Amon,r1i1p1f2,gn,rtmt,201501-210012,v20190731,s3://esgf-world/CMIP6/ScenarioMIP/UA/MCM-UA-1-...
868000,CMIP6,UA,MCM-UA-1-0,ssp585,mon,ocean,Omon,r1i1p1f2,gn,sos,201501-210012,v20190731,s3://esgf-world/CMIP6/ScenarioMIP/UA/MCM-UA-1-...
868001,CMIP6,UA,MCM-UA-1-0,ssp585,mon,ocean,Omon,r1i1p1f2,gn,tos,201501-210012,v20190731,s3://esgf-world/CMIP6/ScenarioMIP/UA/MCM-UA-1-...


In [6]:
#Examples to just search for what we want from the catalog
expname_filter = ['piControl']
table_id_filter = 'Omon'
#model_filter = 'GFDL-ESM4'
variable_id_filter = "thetao"
ens_filter = "r1i1p1f1"
#version_filter = "v20190726"
#grid_label = "gr1"
#cat = col.search(experiment_id=expname_filter, mip_table=table_id_filter,model=model_filter,variable=variable_id_filter,version="v20190726")
cat = col.search(experiment_id=expname_filter,
                 mip_table=table_id_filter,
                 ensemble_member=ens_filter,
                 grid_label='gn',
                 variable=variable_id_filter)

In [7]:
cat

,unique
project,1
institute,8
model,13
experiment_id,1
frequency,1
modeling_realm,1
mip_table,1
ensemble_member,1
grid_label,1
variable,1


In [8]:
cat.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
0,CMIP6,AWI,AWI-CM-1-1-MR,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,240101-241012,v20181218,s3://esgf-world/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/p...
1,CMIP6,AWI,AWI-CM-1-1-MR,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,241101-242012,v20181218,s3://esgf-world/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/p...
2,CMIP6,AWI,AWI-CM-1-1-MR,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,242101-243012,v20181218,s3://esgf-world/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/p...
3,CMIP6,AWI,AWI-CM-1-1-MR,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,243101-244012,v20181218,s3://esgf-world/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/p...
4,CMIP6,AWI,AWI-CM-1-1-MR,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,244101-245012,v20181218,s3://esgf-world/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,CMIP6,NOAA-GFDL,GFDL-ESM4,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,040101-042012,v20180701,s3://esgf-world/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4...
484,CMIP6,NOAA-GFDL,GFDL-ESM4,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,042101-044012,v20180701,s3://esgf-world/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4...
485,CMIP6,NOAA-GFDL,GFDL-ESM4,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,044101-046012,v20180701,s3://esgf-world/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4...
486,CMIP6,NOAA-GFDL,GFDL-ESM4,piControl,mon,ocean,Omon,r1i1p1f1,gn,thetao,046101-048012,v20180701,s3://esgf-world/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4...


In [ ]:
dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 1}},
                                storage_options={'anon':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [ ]:
dset_dict

In [ ]:
exit
# we are not there yet

## Compute Ocean Heat Content

In [ ]:
OHC = 3992 * 1025 * CM4_T['thetao'] * CM4_T['volcello']

In [ ]:
OHC

In [ ]:
global_OHC_upper700m = OHC.sel(lev=slice(0,700)).sum(dim=('lev', 'lat', 'lon'))

In [ ]:
global_OHC_upper2000m = OHC.sel(lev=slice(0,2000)).sum(dim=('lev', 'lat', 'lon'))

In [ ]:
global_OHC_2000below = OHC.sel(lev=slice(2000,7000)).sum(dim=('lev', 'lat', 'lon'))

In [ ]:
global_OHC = OHC.sum(dim=('lev', 'lat', 'lon'))

In [ ]:
global_OHC_level = OHC.sum(dim=('lat', 'lon'))

## Go fast with dask cluster

In dask labextension, create a +NEW cluster. Click the < > widget, scale it to 16 workers and let's go!

In [ ]:
%time _ = global_OHC_upper700m.load()

In [ ]:
# chunks =  (12, 35, 180, 360) -> Wall time: 3min 15s  ! cost of concat netcdf chunks?
# chunks =  (1, 35, 180, 360) -> Wall time: 2min 29s
# chunks =  (1, 1, 180, 360) -> Wall time: 5min 53s ! chunks too small, graph takes forever to build

In [ ]:
%time _ = global_OHC_upper2000m.load()

In [ ]:
%time _ = global_OHC_2000below.load()

In [ ]:
%time _ = global_OHC.load()

In [ ]:
%time _ = global_OHC_level.load()

## Compare with Zanna et al. paper

In [ ]:
Zanna = xr.open_dataset('/net2/rnd/Zanna_2018/OHC_GF_1870_2018.nc')
Zanna = Zanna.rename({'time (starting 1870)': 'time'})
Zanna = Zanna.set_coords(['time'])

In [ ]:
dates = []
for year in Zanna['time'].values:
    dates.append(cftime.DatetimeNoLeap(year, 7, 16, hour=12))

In [ ]:
Zanna['cftime'] = xr.DataArray(np.array(dates), dims='cftime')
Zanna = Zanna.set_coords(['cftime'])

In [ ]:
Zanna

In [ ]:
def anom_yearly_avg(da):
    # compute yearly values
    yearly = da.groupby(da.time.dt.year).mean(dim='time')
    # anomamly to 1870 like in Zanna et al.
    anom = yearly - yearly.sel(year=1870)
    return anom

gOHCanom_upper700m_annual = anom_yearly_avg(global_OHC_upper700m)
gOHCanom_upper2000m_annual = anom_yearly_avg(global_OHC_upper2000m)
gOHCanom_2000below_annual = anom_yearly_avg(global_OHC_2000below)
gOHCanom_annual = anom_yearly_avg(global_OHC)

## Plot the results

In [ ]:
plt.figure()
ax = plt.axes()
(gOHCanom_upper700m_annual / 1e21).plot(ax=ax, label='CM4', color='k')
Zanna['OHC_700m'].plot(ax=ax, label='Zanna', color='r')
plt.legend(fontsize=16)
plt.title('OHC upper 700m')
plt.grid()

In [ ]:
plt.figure()
ax = plt.axes()
(gOHCanom_upper2000m_annual / 1e21).plot(ax=ax, label='CM4', color='k')
Zanna['OHC_2000m'].plot(ax=ax, label='Zanna', color='r')
plt.legend(fontsize=16)
plt.title('OHC upper 2000m')
plt.grid()

In [ ]:
plt.figure()
ax = plt.axes()
(gOHCanom_2000below_annual / 1e21).plot(ax=ax, label='CM4', color='k')
Zanna['OHC_below_2000m'].plot(ax=ax, label='Zanna', color='r')
plt.legend(fontsize=16)
plt.title('OHC below 2000m')
plt.grid()

In [ ]:
plt.figure()
ax = plt.axes()
(gOHCanom_annual / 1e21).plot(ax=ax, label='CM4', color='k')
Zanna['OHC_full_depth'].plot(ax=ax, label='Zanna', color='r')
plt.legend(fontsize=16)
plt.title('OHC full depth')
plt.grid()

### All in one

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=[10,10])

(gOHCanom_upper700m_annual / 1e21).plot(ax=axs[0,0], label='CM4hist', color='k')
Zanna['OHC_700m'].plot(ax=axs[0,0], label='Zanna', color='r')
axs[0,0].legend(fontsize=16)
axs[0,0].set_title('OHC [ZJ] upper 700m')
axs[0,0].set_xlabel("")
axs[0,0].set_ylabel("")
axs[0,0].grid()

(gOHCanom_upper2000m_annual / 1e21).plot(ax=axs[0,1], label='CM4hist', color='k')
Zanna['OHC_2000m'].plot(ax=axs[0,1], label='Zanna', color='r')
axs[0,1].legend(fontsize=16)
axs[0,1].set_title('OHC [ZJ] upper 2000m')
axs[0,1].set_xlabel("")
axs[0,1].set_ylabel("")
axs[0,1].grid()

(gOHCanom_2000below_annual / 1e21).plot(ax=axs[1,0], label='CM4hist', color='k')
Zanna['OHC_below_2000m'].plot(ax=axs[1,0], label='Zanna', color='r')
axs[1,0].legend(fontsize=16)
axs[1,0].set_title('OHC [ZJ] below 2000m')
axs[1,0].set_xlabel("")
axs[1,0].set_ylabel("")
axs[1,0].grid()

(gOHCanom_annual / 1e21).plot(ax=axs[1,1], label='CM4hist', color='k')
Zanna['OHC_full_depth'].plot(ax=axs[1,1], label='Zanna', color='r')
axs[1,1].legend(fontsize=16)
axs[1,1].set_title('OHC [ZJ] full depth')
axs[1,1].set_xlabel("")
axs[1,1].set_ylabel("")
axs[1,1].grid()